In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'CS3540 Final Project/Notebooks'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['extract_filename_features.py', 'extract_audio_features.py', 'data', 'normalize_data.py', 'project_proposal.ipynb', 'load_train_test_split.ipynb', 'svm.ipynb', 'pca_and_svm.ipynb', 'Neural Network.ipynb']


In [ ]:
!pip install audb
!pip install audiofile
!pip install opensmile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.4/167.4 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iso-639: filename=iso_639-0.4.5-py3-none-any.whl size=168840 sha256=5edef05f0efbab15f4a62e7063b83782b17795210c2705a0f3c1f63a0ab97a93
  Stored in directory: /root/.cache/pip/wheels/d8/78/cc/5478ca3b1c3f602eae6f8cdbd78f909c0a0bfa0bbcb5c7771f
Successfully built iso-639
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.2/996.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.2 MB/s eta 0:00:00


In [ ]:
import os
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from six import StringIO
import pydotplus
import numpy as np
import pandas as pd

import audb
import audiofile
import opensmile

import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

In [ ]:
if False:
  # Perform train test split

  filename_df = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "filename_features.csv"))

  # Get list of unique actors

  unique_actors = filename_df.Actor.unique()

  # Train test split 80-20
  train_val, test = train_test_split(unique_actors, test_size=0.2)
  train, val = train_test_split(train_val, test_size=0.25)
  print(train, val, test)

  train_no_val, test_no_val = train_test_split(unique_actors, test_size=0.2)
  print(train_no_val, test_no_val)


In [ ]:
if False:
  actor_list = filename_df["Actor"]

  # Extract features
  start_time = time.time()

  # Select a target list
  label_list = filename_df["Emotion"]
  intensity_list = filename_df["Emotional Intensity"]

  # Initialize Smile - change features as you want
  smile = opensmile.Smile(
      feature_set=opensmile.FeatureSet.eGeMAPSv02,
      feature_level=opensmile.FeatureLevel.Functionals,
  )

  # Iterate and create a new dataframe
  processed_features = []
  for file in filename_df["Filename"]:
      # Read in the file
      signal, sampling_rate = audiofile.read(
          file,
          duration=4,
          always_2d=True,
      )

      # Process file using smile
      signals = smile.process_signal(
          signal,
          sampling_rate
      )
      # Store to list
      processed_features.append(signals)

  # Create new dataframe to save
  # Concatenate the processed features into a single DataFrame
  processed_df = pd.concat([pd.DataFrame(features, columns=list(smile.feature_names)) for features in processed_features], ignore_index=True)

  # Add targets
  processed_df["target"] = label_list
  processed_df["intensity"] = intensity_list

  # Save to csv
  processed_df.to_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "processed_features.csv"))

  # total time
  end_time = time.time()
  print(f"Elapsed time: {end_time - start_time}")

In [ ]:
if False:
  # Normalize data
  # Load data from csv
  data = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "processed_features.csv"))
  data = data.drop(data.columns[0], axis=1)
  features = data.iloc[:, :-2]
  labels = data.iloc[:, -2:]


  # Normalize data
  # Initialize MinMaxScaler
  scaler = MinMaxScaler()

  # Fit and transform the data
  scaled_data = scaler.fit_transform(features)

  # Convert back to DataFrame
  scaled_df = pd.DataFrame(scaled_data, columns=features.columns)

  # Join data with labels
  joined_data = pd.concat([scaled_df, data.iloc[:, -2:]], axis=1)

  joined_data["Actor"] = actor_list

  joined_data.to_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "normalized_features.csv"))

In [ ]:
if False:
  joined_data = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "normalized_features.csv"))
  joined_data["Actor"] = actor_list

  # Retrieving train/test/val data from normalized dataset based on our splits
  train_df = joined_data[joined_data['Actor'].isin(train)].drop(columns=['Actor'])
  val_df = joined_data[joined_data['Actor'].isin(val)].drop(columns=['Actor'])
  test_df = joined_data[joined_data['Actor'].isin(test)].drop(columns=['Actor'])
  train_no_val_df = joined_data[joined_data['Actor'].isin(train_no_val)].drop(columns=['Actor'])
  test_no_val_df = joined_data[joined_data['Actor'].isin(test_no_val)].drop(columns=['Actor'])

In [ ]:
if False:
  train_df.to_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "train.csv"))
  val_df.to_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "val.csv"))
  test_df.to_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "test.csv"))
  train_no_val_df.to_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "train_no_val.csv"))
  test_no_val_df.to_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "test_no_val.csv"))

In [ ]:
def load_train_test():
  # Load data
  train_df = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "train_no_val.csv"))
  test_df = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "test_no_val.csv"))
  # Randomly shuffle data
  train_df = train_df.sample(frac=1)
  test_df = test_df.sample(frac=1)

  return train_df, test_df


def load_train_val_test():
  # Load data
  train_df = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "train.csv"))
  val_df = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "val.csv"))
  test_df = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "test.csv"))
  # Randomly shuffle data
  train_df = train_df.sample(frac=1)
  val_df = val_df.sample(frac=1)
  test_df = test_df.sample(frac=1)

  return train_df, val_df, test_df

def gen_train_test_split():
  GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'CS3540 Final Project/Notebooks'
  GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
  # print(os.listdir(GOOGLE_DRIVE_PATH))
  filename_df = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "filename_features.csv"))

  # Get list of unique actors
  unique_actors = filename_df.Actor.unique()
  train_no_val, test_no_val = train_test_split(unique_actors, test_size=0.2)
  joined_data = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "normalized_features.csv"))
  joined_data["Actor"] = filename_df["Actor"]

  # Retrieving train/test/val data from normalized dataset based on our splits
  train_no_val_df = joined_data[joined_data['Actor'].isin(train_no_val)].drop(columns=['Actor']).sample(frac=1)
  test_no_val_df = joined_data[joined_data['Actor'].isin(test_no_val)].drop(columns=['Actor']).sample(frac=1)

  return train_no_val_df, test_no_val_df


def gen_train_val_test_split():
  GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'CS3540 Final Project/Notebooks'
  GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
  # print(os.listdir(GOOGLE_DRIVE_PATH))

  filename_df = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "filename_features.csv"))

  # Get list of unique actors
  unique_actors = filename_df.Actor.unique()

  train_val, test = train_test_split(unique_actors, test_size=0.2)
  train, val = train_test_split(train_val, test_size=0.25)

  joined_data = pd.read_csv(os.path.join(GOOGLE_DRIVE_PATH, "data", "normalized_features.csv"))
  joined_data["Actor"] = filename_df["Actor"]

  # Retrieving train/test/val data from normalized dataset based on our splits
  train_df = joined_data[joined_data['Actor'].isin(train)].drop(columns=['Actor']).sample(frac=1)
  val_df = joined_data[joined_data['Actor'].isin(val)].drop(columns=['Actor']).sample(frac=1)
  test_df = joined_data[joined_data['Actor'].isin(test)].drop(columns=['Actor']).sample(frac=1)

  return train_df, val_df, test_df


# gen_train_test_split()